In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Flatten
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator
import matplotlib.pyplot as plt
import csv
import os

Import dataset

In [2]:
#==== Import Dataset ====
dataset = pd.read_csv('J1_Atas Bawah.csv')


dataset.dropna(inplace=True)
shuffled_dataset = dataset.sample(frac = 1)

pose = dataset.iloc[:, 6:]   # Input labels (last six columns)
jointsAngle = dataset.iloc[:, :6]   # Output features (first six columns)

shuffled_pose = shuffled_dataset.iloc[:, 6:]   # Input labels (last six columns)
shuffled_jointsAngle = shuffled_dataset.iloc[:, :6]   # Output features (first six columns)

pose_columns = ['x', 'y', 'z', 'roll', 'pitch', 'yaw']  # Replace with your actual column names
jointsAngle_columns = ['theta1', 'theta2', 'theta3', 'theta4']  # Replace with your actual column names

#==== Split Dataset ====
split_index = int(0.8 * len(dataset))


jointsAngle_train = jointsAngle[:split_index]
jointsAngle_test = jointsAngle[split_index:]
pose_train = pose[:split_index]
pose_test = pose[split_index:]

shuffled_jointsAngle_train = shuffled_jointsAngle[:split_index]
shuffled_jointsAngle_test = shuffled_jointsAngle[split_index:]
shuffled_pose_train = shuffled_pose[:split_index]
shuffled_pose_test = shuffled_pose[split_index:]


In [ ]:
# costom loss function
def rmse(y_true, y_pred): #Trial
  return K.sqrt(K.mean(K.square(y_pred-y_true)))

def lms(y_true,y_pred): #by simon haykin book's
  return (K.square(y_pred-y_true))/2

Experiment

In [ ]:

neuron = [20, 50, 100, 200, 300, 400, 500]
learningRate = [0.1, 0.01, 0.001, 0.0001, 0.00001]

for learningRatei in learningRate:
    for neuroni in neuron:
      #Folder for saving the data
      folder_path = '/content/drive/MyDrive/PercobaanTA/DNN/DNN'+'_'+str(learningRatei)+'_2_'+str(neuroni)
      if os.path.exists(folder_path):
          continue
      os.makedirs(folder_path, exist_ok=True)



      #==== DNN Model ====
      model = tf.keras.Sequential()
      model.add(tf.keras.layers.Dense(units=neuroni, activation='relu'))
      #model.add(tf.keras.layers.Dense(units=neuroni, activation='relu'))
      #model.add(tf.keras.layers.LSTM(units=300,  activation='relu', return_sequences=True))
      # Add more RNN layers if needed
      # model.add(...)
      # Add output layer
      model.add(tf.keras.layers.Dense(4, activation="linear"))
      model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learningRatei), loss=lms)

      #==== Target Function ====
      class myCallback(tf.keras.callbacks.Callback):
        def on_epoch_end(self, epoch, logs={}):
          if(logs.get('val_loss') <= 0.00001): # Experiment with changing this value
            print("\nReached loss less than 0.2 so cancelling training!")
            self.model.stop_training = True

      #==== Train Model ====
      callbacks = myCallback()


      #history = model.fit(pose_train, jointsAngle_train, epochs=3000, batch_size=32, callbacks=[callbacks])

      history = model.fit(shuffled_pose_train, shuffled_jointsAngle_train, epochs=10000, batch_size=200, validation_data=(shuffled_pose_test, shuffled_jointsAngle_test), callbacks=[callbacks])

      # Plot loss data
      loss_data = history.history['loss']
      loss_data_plotted = loss_data[1:]

      val_loss_data = history.history['val_loss']
      val_loss_data_plotted = val_loss_data[1:]

      plt.figure()
      plt.plot(range(len(loss_data_plotted)), loss_data_plotted, label='Loss')
      plt.plot(range(len(val_loss_data_plotted)), val_loss_data_plotted, label='Evaluation Loss')
      plt.xlabel('Epoch')
      plt.ylabel('Loss value')
      plt.title('Loss')
      plt.legend()
      # Save loss plot
      plt.savefig(os.path.join(folder_path, 'loss.png'))
      plt.show()

      print(val_loss_data[-1])
      print(loss_data[-1])

      # Write loss and eval loss value
      loss_file = folder_path + '/loss.txt'

      # Open the file in write mode
      with open(loss_file, 'w') as file:
          # Write content to the file
          file.write('Loss = '+str(loss_data[-1])+'\n')
          file.write('Eval loss = '+str(val_loss_data[-1])+'\n')
          file.write('Epoch = '+str(len(loss_data))+'\n')

      #==== Prediction Test ====
      pose_predict = model.predict(pose_test)

      print(pose_predict)

      # Save prediction data into csv file
      header = ['Joint 1', 'Joint 2', 'Joint 3', 'Joint 4']
      rows = pose_predict.tolist()  # Convert predictions to a list
      csv_file = folder_path + '/Predictions_openmanipulatorx.csv'

      with open(csv_file, 'w', newline='') as file:
          writer = csv.writer(file)
          writer.writerow(header)
          writer.writerows(rows)

      i = 0
      jointsAngle_test = jointsAngle[split_index:]
      joint = 1
      # Plot joint angle test data and joint angle prediction
      for column in jointsAngle_test:
          input_data = jointsAngle_test[column]
          predict_data = pose_predict[:, i]
          i = i+1
          plt.figure()
          plt.plot(range(len(input_data)), input_data, label=column)
          plt.plot(range(len(predict_data)), predict_data, label = column + ' prediction')
          plt.xlabel('Index')
          plt.ylabel('Joint Angle (degree)')
          plt.title(column)
          plt.legend()
          # Save plot to folder
          plt.savefig(os.path.join(folder_path, 'A'+str(joint)+'.png'))
          joint = joint+1
          plt.show()



Output hidden; open in https://colab.research.google.com to view.